# Ingest and process data

This notebook loads the data that was stored on Box, and converts it to dataframe and dataset with a common format across all collections.

It provides functionality for downlaoding images.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tools.weaving_tools import NMSCollection, SMGCollection, BTCollection
from pathlib import Path

In [ ]:
nms = NMSCollection(device='mps',img_folder='nms_imgs')
nms.load_original_csvs(files=list(Path('data').glob('NMS_*.*')))
nms.save_csv('data/NMS.csv')

In [ ]:
smg = SMGCollection(device='mps', img_folder='smg_imgs')
smg.load_from_json('data/smg_objects_06_06_2022.json')
smg.save_csv('data/SMG.csv')

In [ ]:
bt = BTCollection(device='mps', img_folder='bt_imgs')
bt.load_from_xml('data/bt_catalogue.xml')
bt.save_csv('data/BT.csv')

# Download images

In [ ]:
collection = SMGCollection(img_folder=Path('smg_imgs'))
collection.load_from_csv('data/SMG.csv')
collection.fetch_images(n=5)
#collection.save_csv('data/SMG.csv')

In [ ]:
collection = BTCollection(img_folder=Path('bt_imgs'))
collection.load_from_csv('data/BT.csv')
collection.fetch_images(n=5)


In [ ]:
collection = NMSCollection(img_folder=Path('nms_imgs'))
collection.load_from_csv('data/NMS.csv')
collection.fetch_images(n=5)


# Create vector database

## Vectorize image and text in each collection

In [ ]:
clip_ckpt = 'clip-ViT-B-32'#'openai/clip-vit-base-patch32'

In [ ]:
nms = NMSCollection(device='mps',img_folder='nms_imgs')
nms.load_from_csv('data/NMS.csv')
nms.vectorize_collection(clip_ckpt)

In [ ]:
smg = SMGCollection(device='mps',img_folder='smg_imgs')
smg.load_from_csv('data/SMG.csv')
# temp to for testing purposes remove later
smg.df = smg.df.sample(frac=0.05).reset_index()
smg.vectorize_collection(clip_ckpt)

In [ ]:
bt = BTCollection(device='mps', img_folder='bt_imgs')
bt.load_from_csv('data/BT.csv')
bt.vectorize_collection(clip_ckpt)

# Create Vector Database

In [ ]:
import chromadb

In [ ]:
client = chromadb.PersistentClient(path="ce_vector_db")

In [ ]:
# try:
#     client.delete_collection(name="congruence_engine")
# except Exception as e:
#     print(e)
    

In [ ]:
collection = client.get_or_create_collection(name="congruence_engine")

In [ ]:
for coll in [nms,bt,smg]:
    for mod in ['image']:
        coll.add_embeddings_to_database(collection,mod)

In [ ]:
collection.count()

In [ ]:
# for coll in [nms,bt,smg]:
#     for mod in ['text']:
#         coll.add_embeddings_to_database(collection,mod)

In [ ]:
# collection.count()

## Fin.

## Query and plot

In [ ]:
import pandas as pd
from tools.weaving_tools import plot_images

In [ ]:
query_embedding = nms.clip_model.encode('machines')

In [ ]:
idx = 3000
query_embedding = np.array(nms.dataset[idx]['clip_image_embedding'])

In [ ]:
results = collection.query(
    query_embeddings=[list(query_embedding.astype(float))],
    n_results=10,
    where={"collection": "nms"},
)

In [ ]:
query_df = pd.DataFrame(results['metadatas'][0])

In [ ]:
Image.open(nms.dataset[idx]['img_path'])

In [ ]:
plot_images(query_df)

# Query

In [ ]:
#idx = 10
query = 'instrument'
#query = 'engine'
query_df = nms.query_collection(query, field='clip_embedding', index_type='clip', k=25)
print(query)
nms.plot_images(query_df)

# Zero Shot Classification

In [ ]:
import transformers
transformers.__version__

In [ ]:
#!pip install -U transformers
# old version '4.25.1'

In [ ]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
checkpoint = 'openai/clip-vit-base-patch32'
model = AutoModelForZeroShotImageClassification.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

In [ ]:
from PIL import Image

In [ ]:
idx = 3000#10
image = Image.open(nms.df.iloc[idx].img_path)

In [ ]:
print(nms.df.iloc[idx].taxonomy)

In [ ]:
import re
#pattern = re.compile()
matches = [re.search(r'[A-Z\-\s]+',str(t)) for t in nms.df.taxonomy.unique()]
matches = set([m.group().strip() for m in matches if m])
candidate_labels = list({i for i in matches if len(i) > 1})

In [ ]:
inputs = processor(images=image, text=candidate_labels, return_tensors="pt", padding=True)

In [ ]:
#inputs

In [ ]:
import torch

with torch.no_grad():
    outputs = model(**inputs)
    
logits = outputs.logits_per_image[0]
probs = logits.softmax(dim=-1).numpy()
scores = probs.tolist()

In [ ]:
result = [
    {"score": score, "label": candidate_label}
        for score, candidate_label in sorted(zip(probs, candidate_labels), key=lambda x: -x[0])
]

In [ ]:
result

In [ ]:
import pandas as pd
pd.Series(scores, index=candidate_labels).plot(kind='bar')

In [ ]:
import torch

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits_per_image[0]
probs = logits.softmax(dim=-1).numpy()
scores = probs.tolist()

result = [
    {"score": score, "label": candidate_label}
        for score, candidate_label in sorted(zip(probs, candidate_labels), key=lambda x: -x[0])
]

result

# Nomic Experiment

In [ ]:
from nomic import atlas
import numpy as np

In [ ]:
%%bash
nomic login pyKINnSOXSivweuAXxprY7j8NtVFI88WC7AeE3BnLAmQZ

In [ ]:
nms_clip_embeddigns = np.array(nms.dataset['clip_image_embedding'])
base_url = 'https://www.nms.ac.uk/search.axd?command=getcontent&server=Detail&value='
nms.df['url'] = nms.df.img_loc.apply(lambda x: base_url + x)
nms_data = [{'category': str(row.taxonomy), 'collection':'nms','id': str(i), 'url': row.url}
            for i, row in nms.df.iterrows()]

In [ ]:
smg_clip_embeddigns = np.array(smg.dataset['clip_image_embedding'])
base_url = 'https://coimages.sciencemuseumgroup.org.uk/images/'
smg.df['url'] = smg.df.img_loc.apply(lambda x: base_url + x)
smg_data = [{'category': str(row.taxonomy), 'collection':'smg','id': str(i), 'url': row.url}
            for i, row in smg.df.iterrows()]

In [ ]:
embeddings = np.concatenate([nms_clip_embeddigns,smg_clip_embeddigns], axis=0)

In [ ]:
data = nms_data + smg_data

In [ ]:
len(data), len(embeddings)

In [ ]:
project = atlas.map_embeddings(embeddings=embeddings,
                                data=data,
                                id_field='id',
                                name='CE Map',
                                colorable_fields=['category','collection'],
                                reset_project_if_exists=True)

In [ ]:
nms.df['tax_simpl'] = nms.df.taxonomy.apply(lambda x: re.search(r'[A-Z\-\s]+', str(x)))
nms.df['tax_simpl'] = nms.df['tax_simpl'].apply(lambda x: x.group() if x else 'OTHER')

In [ ]:
clip_embeddigns.shape, len(data)

In [ ]:
print(project)

In [ ]:
dir(nms)

In [ ]:
nms.load_clip_model()

In [ ]:
from transformers import pipeline

captioner = pipeline("image-to-text",model="google/pix2struct-textcaps-base")

In [ ]:

idx = 302
captioner(nms.df.iloc[idx].img_path)

In [ ]:
Image.open(nms.df.iloc[idx].img_path)

In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset
from PIL import Image
import os
df = nms.df.copy()

# Define a function to load images from file paths
def load_image(image_path):
    return Image.open(image_path)

# Apply the image loading function to the DataFrame
df["image"] = df["img_path"]#.apply(load_image)
df.rename({'description':'text'}, axis=1, inplace=True)
# Create a Hugging Face dataset
dataset = Dataset.from_pandas(df[['text','image']])




In [ ]:
dataset

# Network

In [ ]:
ids2ints = {}

for i,r_id in enumerate(list(nms.df.record_id)):
    ids2ints[r_id] = i
    
for i,r_id in enumerate(list(smg.df.record_id)):
    ids2ints[r_id] = nms.df.shape[0] + i   

In [ ]:
nodes = list(ids2ints.values())

In [ ]:
def result_to_edgelist(results,e1):
    df = pd.DataFrame({k: v[0] for k,v in results.items() if k !='embeddings'} )
    df = pd.concat([df,df.metadatas.apply(pd.Series)], axis=1)
    df['distances'] = (1 / (1+ df.distances)) * 100
    return [(e1,
            ids2ints[row.record_id], 
            row.distances
                        ) for i, row in df.iterrows() 
                if e1 != ids2ints[row.record_id]]

In [ ]:
from tqdm.notebook import tqdm
nodes = []
names = []
edges = []

for i in tqdm(range(1000)): #len(nms.dataset)
    
    
    results_smg = collection.query(
            query_embeddings=[list(np.array(nms.dataset[i]['clip_image_embedding']).astype(float))],
            n_results=3,
            where={"collection": "smg"},
                )
    e1 = ids2ints[nms.dataset[i]['record_id']]
    
    edges.extend(result_to_edgelist(results_smg,e1))
    
    
    results_nms = collection.query(
            query_embeddings=[list(np.array(nms.dataset[i]['clip_image_embedding']).astype(float))],
            n_results=3,
            where={"collection": "nms"},
                )
    
    edges.extend(result_to_edgelist(results_nms,e1))

In [ ]:
nodes = []
for i in edges:
    nodes.extend(i[:2])
nodes = list(set(nodes))

In [ ]:
ints2ids = {v : k for k,v in ids2ints.items()}

In [ ]:
label = [str(ints2ids[n]) for n in nodes] 

In [ ]:
from pyvis.network import Network
#net = Network(notebook=True)
net = Network(height="750px", 
              width="100%", 
              bgcolor="#222222", 
              font_color="white", 
              cdn_resources='remote', 
              #filter_menu=True, 
              notebook=True)

net.add_nodes(
    nodes,
    label=label,
    color=["#00bfff" if i < nms.df.shape[0] else "#ffc0cb" for i in nodes],
)

In [ ]:
net.add_edges(edges)

In [ ]:
#net.show("network.html")

In [ ]:
#net.show_buttons(filter_=['physics'])

In [ ]:
net = draw_network(
    nodes, df, minium_weight=0, repulsion=100, spring_length=500, buttons=["physics"]
)
net.show("match_with_buttons.html")

In [ ]:
net.show_buttons()

In [ ]:
# Image to Text

In [ ]:
!pip install bitsandbytes sentencepiece accelerate transformers

In [ ]:
checkpoint = "HuggingFaceM4/idefics-9b"

In [ ]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

processor = AutoProcessor.from_pretrained(checkpoint)

model = IdeficsForVisionText2Text.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    device_map="auto"
)

In [ ]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes